### Continuing from 3.2.1 ......

We will use XGboost and random forest regression (ensemble) for our future prediction as they have the lowest MAE and highest r2 scores

Use same methadology to predict rents upto 2025

In [1]:
import pandas as pd
import numpy as np
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
df = pd.read_csv('../data/curated/Final_Merged_Dataset.csv')

unwanted_years = [str(i) for i in range(2000, 2012)]

for years in unwanted_years:
    df = df[df.columns.drop(list(df.filter(regex=years)))]
    
immigration_columns = ['cost_text', 'bedrooms', 'bathrooms', 'parking_spaces' ,'2016_immigration',
 '2017_immigration',
 '2018_immigration',
 '2019_immigration',
 '2020_immigration',
 '2021_immigration']

df = df.drop(immigration_columns, axis = 1 )
df = df.groupby('suburb', as_index = False).mean()

df

,suburb,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,2012_median_rent,2013_median_rent,2014_median_rent,2015_median_rent,2016_median_rent,...,2012_unemployment_rate,2013_unemployment_rate,2014_unemployment_rate,2015_unemployment_rate,2016_unemployment_rate,2017_unemployment_rate,2018_unemployment_rate,2019_unemployment_rate,2020_unemployment_rate,2021_unemployment_rate
0,abbotsford,2.934439,1.489185,4.443090,1.067517,450.0,470.0,425.0,440.0,450.0,...,5.3,5.5,5.4,4.5,3.4,4.1,2.9,3.1,4.7,4.5
1,albert park,44.218611,11.892911,5.411578,0.473433,470.0,460.0,480.0,500.0,520.0,...,2.3,2.7,3.2,2.4,2.2,2.1,1.6,2.1,3.6,2.7
2,alfredton,4.955048,6.510578,121.322730,1.698581,270.0,280.0,280.0,285.0,290.0,...,2.8,2.2,2.6,2.6,2.4,2.0,2.2,1.5,3.4,2.2
3,alphington,3.177273,1.260436,9.481345,1.014145,330.0,350.0,350.0,345.0,375.0,...,4.4,5.9,5.7,5.1,4.2,4.7,3.8,4.2,5.8,4.9
4,altona,1.313035,0.931396,18.746987,0.597719,330.0,340.0,350.0,360.0,375.0,...,4.7,4.5,4.7,5.0,5.2,5.7,4.3,3.5,5.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,wendouree,2.355989,2.664463,120.395226,0.755221,270.0,280.0,280.0,285.0,290.0,...,8.9,7.3,8.9,7.7,8.1,6.2,6.9,6.0,9.0,6.2
149,west footscray,2.163598,1.444609,10.739275,1.967266,320.0,320.0,330.0,340.0,360.0,...,6.6,6.2,6.4,6.7,6.7,7.4,5.6,5.2,7.7,5.4
150,williamstown,1.569507,0.901973,14.134295,0.712710,440.0,450.0,440.0,461.0,490.0,...,3.4,3.0,3.7,3.4,3.6,3.9,3.0,2.6,4.2,2.7
151,wodonga,2.278371,4.705043,312.975948,1.161629,265.0,270.0,285.0,290.0,290.0,...,7.0,7.1,6.4,7.5,6.6,4.3,4.8,5.2,4.2,3.9


In [3]:
median_rents = ['suburb', '2012_median_rent', '2013_median_rent', '2014_median_rent', 
                 '2015_median_rent', '2016_median_rent', '2017_median_rent',  
                 '2018_median_rent', '2019_median_rent', '2020_median_rent', 
                 '2021_median_rent']

incomes = ['suburb', '2012_annual_income', '2013_annual_income', '2014_annual_income',
           '2015_annual_income', '2016_annual_income', '2017_annual_income',
           '2018_annual_income', '2019_annual_income', '2020_annual_income', 
           '2021_annual_income']

populations = ['suburb', 'population_2012', 'population_2013', 'population_2014', 
               'population_2015', 'population_2016', 'population_2017', 
               'population_2018', 'population_2019', 'population_2020', 
               'population_2021']

population_density = ['suburb', 'population_density_2012', 'population_density_2013', 'population_density_2014',
                 'population_density_2015', 'population_density_2016',
                 'population_density_2017', 'population_density_2018',
                 'population_density_2019', 'population_density_2020',
                 'population_density_2021']

unemployement = ['suburb', '2012_unemployment_rate', '2013_unemployment_rate', '2014_unemployment_rate',
                 '2015_unemployment_rate', '2016_unemployment_rate', '2017_unemployment_rate', 
                 '2018_unemployment_rate', '2019_unemployment_rate', '2020_unemployment_rate',
                 '2021_unemployment_rate']

In [4]:
melt_rents = pd.melt(df[median_rents], id_vars = 'suburb')
melt_rents['year'] = pd.to_numeric(melt_rents['variable'].str[0:4]) 
melt_rents = melt_rents.drop('variable', axis = 1) 
melt_rents = melt_rents.rename(columns = {'value' : 'median_rent'})

all_rents = melt_rents['median_rent']

melt_rents

,suburb,median_rent,year
0,abbotsford,450.000000,2012
1,albert park,470.000000,2012
2,alfredton,270.000000,2012
3,alphington,330.000000,2012
4,altona,330.000000,2012
...,...,...,...
1525,wendouree,357.208581,2021
1526,west footscray,387.983260,2021
1527,williamstown,516.017635,2021
1528,wodonga,329.083134,2021


In [5]:
melt_income = pd.melt(df[incomes], id_vars = 'suburb')
melt_income['year'] = pd.to_numeric(melt_income['variable'].str[0:4]) 
melt_income = melt_income.drop('variable', axis = 1) 
melt_income = melt_income.rename(columns = {'value' : 'mean_income'})

all_incomes = melt_income['mean_income']

melt_income

,suburb,mean_income,year
0,abbotsford,65938.000000,2012
1,albert park,107915.000000,2012
2,alfredton,52993.000000,2012
3,alphington,68272.000000,2012
4,altona,56677.000000,2012
...,...,...,...
1525,wendouree,50455.929203,2021
1526,west footscray,60293.192444,2021
1527,williamstown,91634.348140,2021
1528,wodonga,54696.818254,2021


In [6]:
melt_population = pd.melt(df[populations], id_vars = 'suburb')
melt_population['year'] = pd.to_numeric(melt_population['variable'].str[11:]) 
melt_population = melt_population.drop('variable', axis = 1) 
melt_population = melt_population.rename(columns = {'value' : 'population'})

all_populations = melt_population['population']

melt_population

,suburb,population,year
0,abbotsford,5544.0,2012
1,albert park,15142.0,2012
2,alfredton,9060.0,2012
3,alphington,8706.0,2012
4,altona,12361.0,2012
...,...,...,...
1525,wendouree,15216.0,2021
1526,west footscray,12800.0,2021
1527,williamstown,17065.0,2021
1528,wodonga,27569.0,2021


In [7]:
melt_population_density = pd.melt(df[population_density], id_vars = 'suburb')
melt_population_density['year'] = pd.to_numeric(melt_population_density['variable'].str[19:]) 
melt_population_density = melt_population_density.drop('variable', axis = 1) 
melt_population_density = melt_population_density.rename(columns = {'value' : 'population_density'})

all_population_densitu = melt_population_density['population_density']

melt_population_density

,suburb,population_density,year
0,abbotsford,3185.291583,2012
1,albert park,3239.138340,2012
2,alfredton,171.880306,2012
3,alphington,3017.363879,2012
4,altona,698.965773,2012
...,...,...,...
1525,wendouree,225.177363,2021
1526,west footscray,2140.217700,2021
1527,williamstown,2322.938078,2021
1528,wodonga,98.879859,2021


In [8]:
melt_unemployement = pd.melt(df[unemployement], id_vars = 'suburb')
melt_unemployement['year'] = pd.to_numeric(melt_unemployement['variable'].str[0:4]) 
melt_unemployement = melt_unemployement.drop('variable', axis = 1) 
melt_unemployement = melt_unemployement.rename(columns = {'value' : 'unemployement'})

all_unemployment = melt_unemployement['unemployement']

melt_unemployement

,suburb,unemployement,year
0,abbotsford,5.3,2012
1,albert park,2.3,2012
2,alfredton,2.8,2012
3,alphington,4.4,2012
4,altona,4.7,2012
...,...,...,...
1525,wendouree,6.2,2021
1526,west footscray,5.4,2021
1527,williamstown,2.7,2021
1528,wodonga,3.9,2021


In [9]:
melted_dfs = [melt_rents, melt_income, melt_population, melt_population_density, melt_unemployement ]
modelling_dataframe = pd.concat(melted_dfs, join='inner', axis=1)
modelling_dataframe = modelling_dataframe.loc[:,~modelling_dataframe.columns.duplicated()].copy()

modelling_dataframe

,suburb,median_rent,year,mean_income,population,population_density,unemployement
0,abbotsford,450.000000,2012,65938.000000,5544.0,3185.291583,5.3
1,albert park,470.000000,2012,107915.000000,15142.0,3239.138340,2.3
2,alfredton,270.000000,2012,52993.000000,9060.0,171.880306,2.8
3,alphington,330.000000,2012,68272.000000,8706.0,3017.363879,4.4
4,altona,330.000000,2012,56677.000000,12361.0,698.965773,4.7
...,...,...,...,...,...,...,...
1525,wendouree,357.208581,2021,50455.929203,15216.0,225.177363,6.2
1526,west footscray,387.983260,2021,60293.192444,12800.0,2140.217700,5.4
1527,williamstown,516.017635,2021,91634.348140,17065.0,2322.938078,2.7
1528,wodonga,329.083134,2021,54696.818254,27569.0,98.879859,3.9


## Step 1 - Create Test Set for Future 

* Use ARMA to determine median_rent, mean_income, population, population_density and unemployement for all suburbs , for the years 2022, 2023, 2024, 2025, 2026


In [10]:
def arma_preds(attribute):
    
    sa2_list = [ i for i in modelling_dataframe['suburb'].unique()]
    full_predictions = pd.DataFrame(columns = ['suburb', attribute , 'year'])
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        for i in sa2_list:

            curr_suburb = modelling_dataframe[modelling_dataframe['suburb'] == i]

            from statsmodels.tsa.statespace.sarimax import SARIMAX

            y = curr_suburb[attribute]

            SARIMAXmodel = SARIMAX(y, order = (1, 0, 1))
            SARIMAXmodel = SARIMAXmodel.fit(disp=0)

            y_pred = SARIMAXmodel.get_forecast(6)
            
            y_pred_df = y_pred.conf_int(alpha = 0.05) 
            y_pred_df[attribute] = SARIMAXmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
            y_pred_df["year"] = [2022, 2023, 2024, 2025, 2026, 2027]
            y_pred_df["suburb"] = i 
            
            if attribute == 'mean_income':
                y_pred_df[attribute] = y_pred_df['upper mean_income']

            full_predictions = pd.concat([full_predictions, y_pred_df[['suburb', attribute , 'year']]])
            
        return full_predictions.reset_index(drop = True)

In [11]:
future_incomes = arma_preds('mean_income')
future_population = arma_preds('population')
future_popden = arma_preds('population_density')
future_unemployment = arma_preds('unemployement')

In [12]:
future_dfs = [future_incomes, future_population, future_popden, future_unemployment]
future_dataframe = pd.concat(future_dfs, join='inner', axis=1)
future_dataframe = future_dataframe.loc[:,~future_dataframe.columns.duplicated()].copy()

future_dataframe

,suburb,mean_income,year,population,population_density,unemployement
0,abbotsford,79318.359206,2022,9783.865576,5621.294297,4.438316
1,abbotsford,82557.279796,2023,9745.402906,5599.184780,4.387938
2,abbotsford,84564.223025,2024,9707.091442,5577.162223,4.338133
3,abbotsford,86146.017737,2025,9668.930589,5555.226285,4.288893
4,abbotsford,87489.882556,2026,9630.919756,5533.376624,4.240211
...,...,...,...,...,...,...
913,yarraville,85668.803193,2023,16451.013238,2927.318918,3.770920
914,yarraville,87265.316700,2024,16447.396334,2926.676287,3.770918
915,yarraville,88526.143837,2025,16443.780225,2926.033798,3.770915
916,yarraville,89599.185470,2026,16440.164912,2925.391449,3.770912


In [13]:
modelling_dataframe = modelling_dataframe.merge(df[['suburb', 'nearest_hospital', 'nearest_train_station',
       'distance_to_cbd', 'nearest_school']], on = 'suburb', how = 'left' )

future_dataframe = future_dataframe.merge(df[['suburb', 'nearest_hospital', 'nearest_train_station',
       'distance_to_cbd', 'nearest_school']], on = 'suburb', how = 'left' )

#### We now have both our train and test set ready ! - Need to Transform Appropiately and Fit Models 

### Step 2 - Applying Appropriate Transformations and Creating Matrices Suitable for The Models 

In [14]:
X_train = modelling_dataframe.drop(['median_rent', 'year'], axis = 1)
X_train = pd.get_dummies(X_train, columns=['suburb'])

y_train = modelling_dataframe['median_rent']

X_train.head()

,mean_income,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,suburb_abbotsford,suburb_albert park,...,suburb_tullamarine,suburb_vermont,suburb_wangaratta,suburb_wantirna,suburb_warragul,suburb_wendouree,suburb_west footscray,suburb_williamstown,suburb_wodonga,suburb_yarraville
0,65938.0,5544.0,3185.291583,5.3,2.934439,1.489185,4.443090,1.067517,1,0,...,0,0,0,0,0,0,0,0,0,0
1,107915.0,15142.0,3239.138340,2.3,44.218611,11.892911,5.411578,0.473433,0,1,...,0,0,0,0,0,0,0,0,0,0
2,52993.0,9060.0,171.880306,2.8,4.955048,6.510578,121.322730,1.698581,0,0,...,0,0,0,0,0,0,0,0,0,0
3,68272.0,8706.0,3017.363879,4.4,3.177273,1.260436,9.481345,1.014145,0,0,...,0,0,0,0,0,0,0,0,0,0
4,56677.0,12361.0,698.965773,4.7,1.313035,0.931396,18.746987,0.597719,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X_test_2022 = future_dataframe[future_dataframe['year'] == 2022].reset_index(drop = True)
X_test_2022 = X_test_2022.drop(['year'], axis = 1)
suburbs_list = X_test_2022['suburb']
X_test_2022 = pd.get_dummies(X_test_2022, columns=['suburb'])

In [16]:
X_test_2023 = future_dataframe[future_dataframe['year'] == 2023].reset_index(drop = True)
X_test_2023 = X_test_2023.drop(['year'], axis = 1)
X_test_2023 = pd.get_dummies(X_test_2023, columns=['suburb'])

In [17]:
X_test_2024 = future_dataframe[future_dataframe['year'] == 2024].reset_index(drop = True)
X_test_2024 = X_test_2024.drop(['year'], axis = 1)
X_test_2024 = pd.get_dummies(X_test_2024, columns=['suburb'])

In [18]:
X_test_2025 = future_dataframe[future_dataframe['year'] == 2025].reset_index(drop = True)
X_test_2025 = X_test_2025.drop(['year'], axis = 1)
X_test_2025 = pd.get_dummies(X_test_2025, columns=['suburb'])

In [19]:
X_test_2026 = future_dataframe[future_dataframe['year'] == 2026].reset_index(drop = True)
X_test_2026 = X_test_2026.drop(['year'], axis = 1)
X_test_2026 = pd.get_dummies(X_test_2026, columns=['suburb'])

In [20]:
X_test_2027 = future_dataframe[future_dataframe['year'] == 2027].reset_index(drop = True)
X_test_2027 = X_test_2027.drop(['year'], axis = 1)
X_test_2027 = pd.get_dummies(X_test_2027, columns=['suburb'])

## Step 3 - The Fun Part - Fitting The Model 

Calculating predicted rents for 2022-2027 

#### XGboost Regression 

In [21]:
from xgboost.sklearn import XGBRegressor

xgboost_regressor = XGBRegressor()
xgboost_regressor.fit(X_train, y_train)

y_pred_2022 = xgboost_regressor.predict(X_test_2022)
y_pred_2023 = xgboost_regressor.predict(X_test_2023)
y_pred_2024 = xgboost_regressor.predict(X_test_2024)
y_pred_2025 = xgboost_regressor.predict(X_test_2025)
y_pred_2026 = xgboost_regressor.predict(X_test_2026)
y_pred_2027 = xgboost_regressor.predict(X_test_2027)

xgboost_predictions = pd.DataFrame(columns = ['Suburb', '2022 Predicted Rent', 
                                            '2023 Predicted Rent', '2024 Predicted Rent',
                                            '2025 Predicted Rent', '2026 Predicted Rent',
                                            '2027 Predicted Rent'])

xgboost_predictions['Suburb'] = suburbs_list
xgboost_predictions['2022 Predicted Rent'] = y_pred_2022
xgboost_predictions['2023 Predicted Rent'] = y_pred_2023
xgboost_predictions['2024 Predicted Rent'] = y_pred_2024
xgboost_predictions['2025 Predicted Rent'] = y_pred_2025
xgboost_predictions['2026 Predicted Rent'] = y_pred_2026
xgboost_predictions['2027 Predicted Rent'] = y_pred_2027

xgboost_predictions

,Suburb,2022 Predicted Rent,2023 Predicted Rent,2024 Predicted Rent,2025 Predicted Rent,2026 Predicted Rent,2027 Predicted Rent
0,abbotsford,497.688965,522.485413,518.077332,518.077332,531.016418,531.016418
1,albert park,552.319214,532.987732,537.693787,536.262146,536.262146,536.262146
2,alfredton,372.218994,374.568787,408.667297,408.667297,408.667297,408.667297
3,alphington,406.877350,412.279083,412.279083,412.279083,412.279083,412.279083
4,altona,420.777618,414.271790,414.646149,426.960907,427.531464,427.531464
...,...,...,...,...,...,...,...
148,wendouree,339.344910,335.708374,346.656311,363.437225,356.852112,356.852112
149,west footscray,379.167267,377.765472,392.516998,392.049530,393.784088,400.137024
150,williamstown,504.787842,506.743896,510.728485,500.960907,539.143982,544.132141
151,wodonga,323.677551,323.896454,323.167023,319.403625,319.403625,319.403625


In [22]:
from sklearn.ensemble import RandomForestRegressor


randomforest_regressor = RandomForestRegressor()
randomforest_regressor.fit(X_train, y_train)

y_pred_2022 = randomforest_regressor.predict(X_test_2022)
y_pred_2023 = randomforest_regressor.predict(X_test_2023)
y_pred_2024 = randomforest_regressor.predict(X_test_2024)
y_pred_2025 = randomforest_regressor.predict(X_test_2025)

rforest_predictions =  pd.DataFrame(columns = ['Suburb', '2022 Predicted Rent', 
                                            '2023 Predicted Rent', '2024 Predicted Rent',
                                            '2025 Predicted Rent', '2026 Predicted Rent',
                                            '2027 Predicted Rent'])

rforest_predictions['Suburb'] = suburbs_list

rforest_predictions['2022 Predicted Rent'] = y_pred_2022
rforest_predictions['2023 Predicted Rent'] = y_pred_2023
rforest_predictions['2024 Predicted Rent'] = y_pred_2024
rforest_predictions['2025 Predicted Rent'] = y_pred_2025
rforest_predictions['2026 Predicted Rent'] = y_pred_2026
rforest_predictions['2027 Predicted Rent'] = y_pred_2027

rforest_predictions

,Suburb,2022 Predicted Rent,2023 Predicted Rent,2024 Predicted Rent,2025 Predicted Rent,2026 Predicted Rent,2027 Predicted Rent
0,abbotsford,494.874275,526.946740,522.978942,521.434402,531.016418,531.016418
1,albert park,544.181221,544.181221,544.181221,544.181221,536.262146,536.262146
2,alfredton,393.566042,395.342452,408.362829,408.212829,408.667297,408.667297
3,alphington,417.442531,419.771372,421.836072,422.962618,412.279083,412.279083
4,altona,406.208481,414.356576,421.337671,428.654014,427.531464,427.531464
...,...,...,...,...,...,...,...
148,wendouree,319.404455,318.374455,315.847364,316.627364,356.852112,356.852112
149,west footscray,390.085355,394.540664,400.922744,401.772744,393.784088,400.137024
150,williamstown,500.598251,500.778251,498.261002,500.873791,539.143982,544.132141
151,wodonga,316.009097,308.682446,308.893127,308.893127,319.403625,319.403625


## Answering research question 3 - What are the top 10 suburbs with the highest predicted growth rate ?

### Calculating Average YoY percentage change

In [23]:
xgb_melt = xgboost_predictions.melt('Suburb')
xgb_melt['Year'] = xgb_melt['variable'].str[0:4].astype(int)

xgb_melt = xgb_melt.rename(columns = {'value' : 'Predicted Rent'})
xgb_melt = xgb_melt.drop('variable', axis = 1).sort_values(by = ['Suburb', 'Year']).reset_index(drop = True)

percentage_changes_xgb= xgb_melt.copy()

percentage_changes_xgb.sort_values(['Suburb','Year'], inplace = True, ascending=[True, True])

percentage_changes_xgb['pct_chg'] = (percentage_changes_xgb.groupby('Suburb')['Predicted Rent']
                                  .apply(pd.Series.pct_change)) * 100

prc_changes_xgb_mean = percentage_changes_xgb.groupby(['Suburb'], as_index = False).mean().sort_values(by =['pct_chg'], ascending = False)

ten_highest_xgb = prc_changes_xgb_mean.head(10).reset_index(drop = True)[['Suburb', 'pct_chg']]

ten_highest_xgb = ten_highest_xgb.rename(columns = {'Suburb' : 'suburb'})

ten_highest_xgb 

,suburb,pct_chg
0,horsham,6.034717
1,moe,3.904290
2,seymour,3.540180
3,hampton park,3.258965
4,sale,2.971604
5,parkville,2.967781
6,carlton,2.626669
7,lara,2.590163
8,mount eliza,2.485652
9,sunshine,2.423167


In [24]:
ten_highest_xgb['pct_chg'] = np.log(ten_highest_xgb['pct_chg'])

In [25]:
rf_melt =rforest_predictions.melt('Suburb')
rf_melt['Year'] = rf_melt['variable'].str[0:4].astype(int)

rf_melt = rf_melt.rename(columns = {'value' : 'Predicted Rent'})
rf_melt = rf_melt.drop('variable', axis = 1).sort_values(by = ['Suburb', 'Year']).reset_index(drop = True)

percentage_changes_rf= rf_melt.copy()

percentage_changes_rf.sort_values(['Suburb','Year'], inplace = True, ascending=[True, True])

percentage_changes_rf['pct_chg'] = (percentage_changes_rf.groupby('Suburb')['Predicted Rent']
                                  .apply(pd.Series.pct_change)) * 100

prc_changes_rf_mean = percentage_changes_rf.groupby(['Suburb'], as_index = False).mean().sort_values(by =['pct_chg'], ascending = False)

ten_highest_rf = prc_changes_rf_mean.head(10).reset_index(drop = True)[['Suburb', 'pct_chg']]
ten_highest_rf = ten_highest_rf.rename(columns = {'Suburb' : 'suburb'})
ten_highest_rf 

,suburb,pct_chg
0,horsham,5.306349
1,moe,4.050202
2,hampton park,3.916615
3,traralgon,3.741030
4,geelong,3.723551
5,geelong west,3.310318
6,lara,3.006906
7,parkville,2.817600
8,newcomb,2.676166
9,sale,2.653831


In [26]:
ten_highest_rf['pct_chg'] = np.log(ten_highest_rf['pct_chg'])
ten_highest_rf['geo_filler'] = 1 

## Plotting results geospatially 

#### Geospatial helper function

In [27]:
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.features import GeoJsonTooltip

import geopandas as gpd
from shapely.geometry import Point
import folium
from shapely import wkt 

vic_sf = gpd.read_file('../data/raw/vic_shapefile_zipped.zip')

columns = ['suburb', '2021_median_rent', 'population_2021', 'nearest_hospital', 
           'nearest_hospital', 'nearest_train_station', 'distance_to_cbd',
           '2021_unemployment_rate', 'population_density_2021', 
           '2021_annual_income']

combined = pd.read_csv('../data/curated/Final_Merged_Dataset.csv')
combined = combined[columns]

suburbs = combined.groupby('suburb', as_index = False).mean().dropna(how = 'any').reset_index(drop = True)
suburbs = suburbs.reset_index(drop = True).T.drop_duplicates().T

def preproc_shapefile(vic_shapefile):
    
    #casefold into lower cases 
    vic_shapefile['LOC_NAME'] = vic_shapefile['LOC_NAME'].str.lower()
    
    #fix up some suburb names
    suburbs.loc[suburbs['suburb'] == 'prahran east','suburb'] = 'prahran'
    suburbs.loc[suburbs['suburb'] == 'preston west','suburb'] = 'preston'
    suburbs.loc[suburbs['suburb'] == 'sanctuary lakes','suburb'] = 'point cook'
    
    #remove duplicates
    vic_shapefile_no_duplicates = vic_shapefile.loc[vic_shapefile['LOC_NAME'].drop_duplicates().index,:]
    
    vic_shapefile['centroid'] = vic_shapefile['geometry'].map(
    lambda polygon: 
        wkt.loads( #after finding centroid, need to re-parse this back into a Point object
            wkt.loads(str(polygon)).centroid.wkt #need str object to find centroid
        )
    )
    
    return vic_shapefile

vic_shapefile_no_duplicates = preproc_shapefile(vic_sf)


def geopandas_plotting(rental_df, 
                       sf, 
                       legend_name='Percentage Change in Rent', 
                       col_to_be_plot='cost_text',
                       save=False,
                       saved_map_name='',
                       map_format='html', fill_color = 'Reds'):
    
    
    tooltip = GeoJsonTooltip(
    fields=["col_to_be_plot"],
    labels=True
    )
    
    """
    Helper function to plot an aggregated metric by suburb
    (e.g. taking the mean of rental prices per suburb)
    
    Arguments:
    1. rental_df: df containing suburb columns and an aggregated
    metric corresponding to each suburb 
    
    E.g. columns may look like SUBURB || MEAN OF RENTAL PRICES
    
    2. sf: preprocessed victoria shapefile using the function above
    
    3. legend_name: legend text to appear on bottom right of the folium map
    
    4. col_to_be_plot: which numeric col to be plotted 
    
    
    """
    
    #if not specifying the legend name then the default one 
    #here will be used. 
    if not legend_name:
        legend_name = f'Distribution of {col_to_be_plot} across suburbs'

    #sf: shapefile of all victorian suburbs (vic_shapefile_no_duplicates)

    #need to have shapefile and zone lookups (in this case, suburb) before plotting
    

    # Convert the geometry shaape to to latitude and longitude
    # Please attribute this if you are using it
    sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

    #gdf = rental_df merged with sf to obtain the geometry for each subrub
    gdf = rental_df.merge(sf[['LOC_NAME', 'geometry']],
            left_on = 'suburb',
            right_on = 'LOC_NAME',
            how='inner').\
            drop(columns=['LOC_NAME'])

    #convert the merged rental_df to a GeoDataFrame 
    gdf = gpd.GeoDataFrame(gdf)

    # combine geopandas with folium to create visualistion,  
    # but still need geojson and folium map object

    #note: drop duplicates because initiallly there are multiple properties in a suburb
    #but to find the geometry we don't require all duplicates. 
    geoJSON = gdf[['suburb', 'geometry']].drop_duplicates('suburb').to_json()

    m = folium.Map(location=[-37.7829335, 144.9575056])

    # refer to the folium documentations on more information on how to plot aggregated data.
    c = folium.Choropleth(
        geo_data=geoJSON, # geoJSON 
        name='choropleth', # name of plot
        data=gdf, # data source
        columns=['suburb',col_to_be_plot], # the columns required
        key_on='properties.suburb', # this is from the geoJSON's properties
        nan_fill_color='black',
        fill_color = fill_color,
        legend_name=legend_name,
        tool_tip = tooltip 
    )

    c.add_to(m)
    display(m)
    
    if save == True:
        m.save(f'../plots/{saved_map_name}.{map_format}')

## Plotting suburbs with highest predicted increase in rent using XGboost Regression 

In [28]:
geopandas_plotting(
     ten_highest_xgb , 
     vic_shapefile_no_duplicates, 
     col_to_be_plot='pct_chg',
     map_format='html',
     save = True
     
 )

## Plotting subrubs with highest predicted increase in rent using Random Forest Regression

In [31]:
geopandas_plotting(
     ten_highest_rf , 
     vic_shapefile_no_duplicates, 
     col_to_be_plot='geo_filler',
     map_format='html',
     save=True,
     saved_map_name='Highest_Predicted_Growth_Suburbs ',
 )